In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics import mean_absolute_error
import os

In [2]:
cuda_av = False
if torch.cuda.is_available():
    cuda_av = True

torch.manual_seed(0)
np.random.seed(0)

In [3]:

class CustomRNN(nn.Module):
    def __init__(self, cell_type, hidden_size, num_layers, bidirectional):
        super(CustomRNN, self).__init__()
        torch.manual_seed(0)

        if bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        if cell_type == "RNN":
            self.rnn = nn.RNN(input_size=1, hidden_size=hidden_size,
                              num_layers=num_layers, batch_first=True,
                              bidirectional=bidirectional)
        elif cell_type == "GRU":
            self.rnn = nn.GRU(input_size=1, hidden_size=hidden_size,
                              num_layers=num_layers, batch_first=True,
                              bidirectional=bidirectional)
        else:
            self.rnn = nn.LSTM(input_size=1, hidden_size=hidden_size,
                               num_layers=num_layers, batch_first=True,
                               bidirectional=bidirectional)

        self.linear = nn.Linear(hidden_size * self.num_directions, 1)
        self.act = nn.ReLU()

    def forward(self, x):
        pred, hidden = self.rnn(x, None)
        pred = self.linear(pred).view(pred.data.shape[0], -1, 1)
        pred = torch.min(pred, x)
        return pred

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=7, stride=1, padding=2)
        self.bn1 = nn.BatchNorm2d(20)

        self.conv2 = nn.Conv2d(20, 16, kernel_size=2, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(16, 64, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.ConvTranspose2d(64, 16, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(16)

        self.conv5 = nn.ConvTranspose2d(16, 6, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(6)

        self.conv6 = nn.ConvTranspose2d(6, 1, kernel_size=5, stride=1, padding=2) 
        
        self.act = nn.ReLU()
        
    # forward method
    def forward(self, input):
        
        e1 = self.conv1(input)
        bn1 = self.bn1(self.act(e1))
        e2 = self.bn2(self.conv2(bn1))        
        e5 = self.bn5(self.conv5(e2))
        e6 = self.conv6(e5)
        return e6

In [4]:
def preprocess(train, valid, test):
    out_train = [None for temp in range(len(ORDER))]
    for a_num, appliance in enumerate(ORDER):
        out_train[a_num] = Variable(
            torch.Tensor(train[:, APPLIANCE_ORDER.index(appliance), :, :].reshape((train.shape[0], 1, -1, 24))))
        if cuda_av:
            out_train[a_num] = out_train[a_num].cuda()

    out_valid = [None for temp in range(len(ORDER))]
    for a_num, appliance in enumerate(ORDER):
        out_valid[a_num] = Variable(
            torch.Tensor(valid[:, APPLIANCE_ORDER.index(appliance), :, :].reshape((valid.shape[0], 1, -1, 24))))
        if cuda_av:
            out_valid[a_num] = out_valid[a_num].cuda()
            
    out_test = [None for temp in range(len(ORDER))]
    for a_num, appliance in enumerate(ORDER):
        out_test[a_num] = Variable(
            torch.Tensor(test[:, APPLIANCE_ORDER.index(appliance), :, :].reshape((test.shape[0], 1, -1, 24))))
        if cuda_av:
            out_test[a_num] = out_test[a_num].cuda()

    return out_train, out_valid, out_test

In [5]:
dataset = 1
fold_num = 1
num_folds = 5
train, test = get_train_test(dataset, num_folds=num_folds, fold_num=fold_num)
valid = train[int(0.8*len(train)):].copy()
train = train[:int(0.8 * len(train))].copy()
train_aggregate = train[:, 0, :, :].reshape(train.shape[0], 1, -1, 24)
valid_aggregate = valid[:, 0, :, :].reshape(valid.shape[0], 1, -1, 24)
test_aggregate = test[:, 0, :, :].reshape(test.shape[0], 1, -1, 24)

In [6]:
ORDER = ['fridge', 'dr', 'hvac', 'dw', 'mw']
out_train, out_valid, out_test = preprocess(train, valid, test)

In [7]:
inp = Variable(torch.Tensor(train_aggregate), requires_grad=False)
valid_inp = Variable(torch.Tensor(valid_aggregate), requires_grad=False)
test_inp = Variable(torch.Tensor(test_aggregate), requires_grad=False)
if cuda_av:
    inp = inp.cuda()
    valid_inp = valid_inp.cuda()
    test_inp = test_inp.cuda()
valid_out = torch.cat([out_valid[appliance_num] for appliance_num, appliance in enumerate(ORDER)])
test_out = torch.cat([out_test[appliance_num] for appliance_num, appliance in enumerate(ORDER)])
train_out = torch.cat([out_train[appliance_num] for appliance_num, appliance in enumerate(ORDER)])

In [8]:
class AppliancesRNNCNN(nn.Module):
    def __init__(self, cell_type, hidden_size, num_layers, bidirectional, num_appliance):
        super(AppliancesRNNCNN, self).__init__()
        self.num_appliance = num_appliance
        self.preds = {}
        self.order = ORDER
        for appliance in range(self.num_appliance):
            if ORDER[appliance] in ['hvac', 'fridge']:
                print("use RNN")
                if cuda_av:
                    setattr(self, "Appliance_" + str(appliance), CustomRNN(cell_type, hidden_size,
                                                                           num_layers, bidirectional).cuda())
                else:
                    setattr(self, "Appliance_" + str(appliance), CustomRNN(cell_type, hidden_size,
                                                                           num_layers, bidirectional))
            else:
                print("use CNN")
                if cuda_av:
                    setattr(self, "Appliance_" + str(appliance), CustomCNN().cuda())
                else:
                    setattr(self, "Appliance_" + str(appliance), CustomCNN())


    def forward(self, *args):
        agg_current = args[0]
        flag = False
        if np.random.random() > args[1]:
            flag = True
        else:
            pass
        for appliance in range(self.num_appliance):
            agg_current = agg_current.contiguous()
            if ORDER[appliance] in ['hvac', 'fridge']:
                agg_current = agg_current.view(agg_current.shape[0], -1, 1)
            else:
                agg_current = agg_current.view(agg_current.shape[0], 1, -1, 24)
            
            self.preds[appliance] = getattr(self, "Appliance_" + str(appliance))(agg_current)
            
            agg_current = agg_current.view(agg_current.shape[0], 1, -1, 24)
            self.preds[appliance] = self.preds[appliance].view(self.preds[appliance].shape[0], 1, -1, 24)
            
            
            if flag:
                agg_current = agg_current - self.preds[appliance]
            else:
                agg_current = agg_current - args[2 + appliance]

        return torch.cat([self.preds[a] for a in range(self.num_appliance)])

In [9]:
loss_func = nn.L1Loss()
model = AppliancesRNNCNN('GRU', 50, 2, True, len(ORDER))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
p=0
params = [inp, p]

if cuda_av:
    model = model.cuda()
    loss_func = loss_func.cuda()
    
for a_num, appliance in enumerate(ORDER):
    params.append(out_train[a_num])

if cuda_av:
    train_out = train_out.cuda()

use CNN
use RNN
use RNN
use CNN
use CNN


In [ ]:
for k in range(4000):
    pred = model(*params)
    optimizer.zero_grad()
    loss = loss_func(pred, train_out)
    
    
    
    if cuda_av:
        test_inp = test_inp.cuda()
    test_params = [test_inp, -2]
    for i in range(len(ORDER)):
        test_params.append(None)
    test_pr = model(*test_params)
    test_loss = loss_func(test_pr, test_out)
    
    print(k, loss.data[0], test_loss.data[0])

    loss.backward()
    optimizer.step()

0 39.79768371582031 50.715614318847656
1 39.78328323364258 50.57164001464844
2 39.761749267578125 50.69451141357422
3 39.74970245361328 50.75785446166992
4 39.735313415527344 50.810813903808594
5 39.69281768798828 50.751094818115234
6 39.679351806640625 50.8043212890625
7 39.74676513671875 50.60640335083008
8 39.74199676513672 50.79082107543945
9 39.71002197265625 50.89620590209961
10 39.73906707763672 50.675662994384766
11 39.70842361450195 50.635986328125
12 39.685211181640625 50.686038970947266
13 39.65639877319336 50.85262680053711
14 40.056365966796875 50.320438385009766
15 39.836082458496094 50.538856506347656
16 40.03162384033203 51.63629913330078
17 39.901519775390625 51.45805740356445
18 39.86384582519531 50.570762634277344
19 39.971168518066406 50.479331970214844
20 39.6970100402832 51.01506042480469
21 39.910465240478516 51.314849853515625
22 39.71820068359375 50.54423522949219
23 39.861507415771484 50.236942291259766
24 39.79844284057617 50.20582580566406
25 39.718826293945

206 38.781707763671875 48.8636360168457
207 38.77463912963867 48.86792755126953
208 38.75998306274414 48.83001708984375
209 38.760684967041016 48.94241714477539
210 38.76028060913086 48.8320426940918
211 38.7426872253418 48.88446044921875
212 38.751136779785156 48.93061828613281
213 38.760250091552734 48.822750091552734
214 38.752750396728516 48.86294174194336
215 38.80889129638672 49.06700134277344
216 38.77944564819336 48.7044677734375
217 38.75968933105469 48.73820495605469
218 38.84196472167969 49.08711242675781
219 38.76100540161133 48.65956115722656
220 38.77619552612305 48.6448860168457
221 38.80004119873047 49.04872512817383
222 38.72758483886719 48.87642288208008
223 38.781494140625 48.56551742553711
224 38.72853469848633 48.7703971862793
225 38.77311325073242 48.97548294067383
226 40.16291809082031 49.731117248535156
227 39.51485061645508 48.99440002441406
228 39.63370132446289 48.26299285888672
229 40.18051528930664 48.35436248779297
230 39.74372863769531 48.56155776977539
2

410 38.2194709777832 48.148258209228516
411 38.24100112915039 47.9892578125
412 38.223201751708984 48.08863830566406
413 38.20832824707031 48.24310302734375
414 38.207645416259766 48.2973518371582
415 38.19523620605469 48.16563034057617
416 38.17512512207031 48.16184997558594
417 38.18088150024414 48.15742111206055
418 38.1628303527832 48.129539489746094
419 38.158817291259766 48.17330551147461
420 38.17283248901367 48.38985824584961
421 38.146080017089844 48.1450309753418
422 38.15633773803711 48.08646774291992
423 38.136192321777344 48.223167419433594
424 38.11427688598633 48.19265365600586
425 38.124351501464844 47.96407699584961
426 38.11216735839844 48.02462387084961
427 38.1118278503418 48.154624938964844
428 38.09535217285156 48.025630950927734
429 38.09026336669922 47.97005844116211
430 38.078060150146484 48.12788772583008
431 38.0869026184082 47.97892379760742
432 38.065067291259766 48.04133224487305
433 38.0677604675293 48.173702239990234
434 38.05124282836914 48.058986663818

613 38.23997116088867 48.38224411010742
614 38.18611145019531 48.08335494995117
615 38.18595886230469 47.894813537597656
616 38.151302337646484 47.92594528198242
617 38.25449752807617 48.32221984863281
618 38.37269592285156 48.302833557128906
619 38.39710998535156 48.12592315673828
620 38.45766067504883 48.049617767333984
621 38.53377914428711 48.03269577026367
622 38.5617561340332 48.057003021240234
623 38.562374114990234 47.17237854003906
624 38.646087646484375 47.2802619934082
625 38.56296157836914 47.083919525146484
626 38.61040496826172 46.87226867675781
627 38.57467269897461 46.92098617553711
628 38.531166076660156 47.23373794555664
629 38.512176513671875 47.21135330200195
630 38.45014953613281 47.17363357543945
631 38.39898681640625 47.034523010253906
632 38.32847595214844 46.85841369628906
633 38.28773880004883 47.2380256652832
634 38.24151611328125 47.807430267333984
635 38.21395492553711 47.74641418457031
636 38.210445404052734 47.36509704589844
637 38.1746826171875 47.192741

816 37.756282806396484 45.789588928222656
817 37.7294807434082 45.880638122558594
818 37.719276428222656 46.13801956176758
819 37.72201156616211 46.2852897644043
820 37.77241516113281 46.203392028808594
821 37.76557922363281 46.047332763671875
822 37.75270080566406 45.950462341308594
823 37.7220458984375 45.91666793823242
824 37.70851516723633 45.980648040771484
825 37.7053108215332 46.03730773925781
826 37.70082473754883 45.99527359008789
827 37.674686431884766 45.834171295166016
828 37.670108795166016 45.74917984008789
829 37.6365852355957 45.72897720336914
830 37.597312927246094 45.8675537109375
831 37.58539962768555 46.063232421875
832 37.557411193847656 46.14111328125
833 37.5510368347168 46.083770751953125
834 37.56678771972656 46.044761657714844
835 37.55548095703125 46.09617233276367
836 37.54751205444336 46.177494049072266
837 37.56547927856445 46.09987258911133
838 37.53812026977539 46.07271957397461
839 37.49268341064453 46.06395721435547
840 37.47574234008789 46.12620162963

...

3003 35.930301666259766 43.54893112182617
3004 35.95269012451172 43.34597396850586
3005 35.887962341308594 43.39466857910156
3006 35.87776565551758 43.63867950439453
3007 35.87705993652344 43.80796813964844
3008 35.88130187988281 43.55537033081055
3009 35.82400131225586 43.77302932739258
3010 35.75810241699219 43.569332122802734
3011 35.70024871826172 43.47529602050781
3012 35.672908782958984 43.392234802246094
3013 35.66107940673828 43.519596099853516
3014 36.11448669433594 42.7207145690918
3015 36.09840393066406 42.75007629394531
3016 35.856266021728516 43.0079460144043
3017 35.72261047363281 43.4086799621582
3018 35.95764923095703 43.8851203918457
3019 35.93354415893555 43.77692413330078
3020 35.78312683105469 43.37669372558594
3021 35.68559646606445 42.9576416015625
3022 35.74338912963867 42.77688217163086
3023 35.60774612426758 42.886314392089844
3024 35.59056854248047 43.2057991027832
3025 35.61288070678711 43.273521423339844
3026 35.53659439086914 43.09947204589844
3027 35.52643

3202 36.39765548706055 44.508296966552734
3203 36.47353744506836 44.89423751831055
3204 36.23884582519531 44.593929290771484
3205 36.67281723022461 44.34667205810547
3206 36.40840148925781 44.36580276489258
3207 37.26424789428711 46.07711410522461
3208 37.25202560424805 46.5430793762207
3209 36.66984176635742 44.52305603027344
3210 37.762325286865234 43.853607177734375
3211 37.83373260498047 43.73353576660156
3212 37.69580841064453 43.74300003051758
3213 37.081016540527344 43.91747283935547
3214 37.2760009765625 45.3709716796875
3215 37.72994613647461 45.2733039855957
3216 37.739959716796875 45.65843963623047
3217 37.33625793457031 45.17192840576172
3218 37.0584602355957 44.15314483642578
3219 37.391090393066406 43.632179260253906
3220 37.47188186645508 44.302589416503906
3221 37.15006637573242 44.399967193603516
3222 36.87560272216797 45.09640884399414
3223 37.05705261230469 45.68402862548828
3224 37.021820068359375 45.68268585205078
3225 36.80229187011719 45.28184509277344
3226 36.62

3400 35.44096755981445 44.60430908203125
3401 35.44609451293945 44.56135559082031
3402 35.409080505371094 44.736228942871094
3403 35.41630172729492 44.91731643676758
3404 35.40279769897461 44.908538818359375
3405 35.37938690185547 44.73857498168945
3406 36.60301971435547 44.3093147277832
3407 37.17087173461914 44.654571533203125
3408 36.26948547363281 44.38298797607422
3409 36.315582275390625 45.037723541259766
3410 36.700408935546875 45.67925262451172
3411 37.133968353271484 46.087379455566406
3412 36.896305084228516 45.87765121459961
3413 36.6124267578125 45.71685791015625
3414 36.468467712402344 45.4416389465332
3415 36.47966003417969 45.1146240234375
3416 36.48594284057617 44.714881896972656
3417 36.26914978027344 44.668521881103516
3418 36.3233757019043 44.970130920410156
3419 36.37966537475586 45.527400970458984
3420 36.34025573730469 45.275691986083984
3421 36.18030548095703 45.017555236816406
3422 36.158756256103516 44.67641830444336
3423 36.18733215332031 44.76868438720703
342

3599 35.05258560180664 43.045501708984375
3600 35.04361343383789 43.004425048828125
3601 35.03584671020508 42.96205520629883
3602 35.029598236083984 42.980281829833984
3603 35.03425216674805 42.960594177246094
3604 35.03175354003906 43.00668716430664
3605 35.032962799072266 42.95521545410156
3606 35.07239532470703 43.19971466064453
3607 35.12580108642578 43.503971099853516
3608 35.196632385253906 42.858577728271484
3609 35.14979934692383 44.646270751953125
3610 35.162010192871094 44.83854293823242
3611 35.16889953613281 44.47412109375
3612 35.136653900146484 45.0064697265625
3613 35.18131637573242 45.2066535949707
3614 35.1175537109375 44.721683502197266
3615 35.138648986816406 44.301002502441406
3616 35.13623809814453 44.58506774902344
3617 35.08909606933594 44.588478088378906
3618 35.09605026245117 42.91505813598633
3619 35.076961517333984 43.0097541809082
3620 35.09694290161133 43.04542541503906
3621 35.093589782714844 42.80479431152344
3622 35.08768844604492 42.78373336791992
3623 

3798 34.924781799316406 42.69367218017578
3799 35.06382369995117 45.12726593017578
3800 35.460182189941406 45.454463958740234
3801 35.763938903808594 45.319068908691406
3802 36.224273681640625 45.22528076171875
3803 36.271522521972656 45.595611572265625
3804 36.45436477661133 46.31289291381836
3805 36.60586166381836 46.2822380065918
3806 36.75462341308594 45.93239212036133
3807 36.790550231933594 45.70922088623047
3808 36.851375579833984 45.904335021972656
3809 36.89069747924805 45.82808303833008
3810 36.90908432006836 45.960208892822266
3811 36.91318130493164 46.446746826171875
3812 36.791316986083984 45.84463882446289
3813 36.781410217285156 44.98701477050781
3814 36.70475387573242 44.025665283203125
3815 36.732120513916016 44.79462814331055
3816 36.581966400146484 44.256526947021484
3817 36.5181999206543 44.10581588745117
3818 36.44422912597656 44.07832717895508
3819 36.3243408203125 44.204872131347656
3820 36.27534484863281 44.283935546875
3821 36.20750427246094 44.323822021484375


In [ ]:
test_pred = torch.split(test_pr, test_aggregate.shape[0])
test_fold = [None for x in range(len(ORDER))]
if cuda_av:
    for appliance_num, appliance in enumerate(ORDER):
        test_fold[appliance_num] = test_pred[appliance_num].cpu().data.numpy().reshape(-1, 24)
else:
    for appliance_num, appliance in enumerate(ORDER):
        test_fold[appliance_num] = test_pred[appliance_num].data.numpy().reshape(-1, 24)

In [ ]:
test_gt_fold = [None for x in range(len(ORDER))]
for appliance_num, appliance in enumerate(ORDER):
    test_gt_fold[appliance_num] = test[:, APPLIANCE_ORDER.index(appliance), :, :].reshape(
        test_aggregate.shape[0],
        -1, 1).reshape(-1, 24)

In [ ]:
test_error = {}

for appliance_num, appliance in enumerate(ORDER):
    test_error[appliance] = mean_absolute_error(test_fold[appliance_num], test_gt_fold[appliance_num])

In [ ]:
test_error

In [32]:
test_error

{'dr': 51.895019602707926,
 'dw': 11.02489129080851,
 'fridge': 28.159192659996876,
 'hvac': 285.86724458947918,
 'mw': 7.4520018024466692}